# Problem with R² Score

The **R² score** measures how well a regression model explains the variability of the target variable.

$$
R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}
$$

However, one major problem with R² is that **it always increases (or stays the same) when new independent variables are added to the model**, even if those variables have **no real relationship** with the target.

This happens because R² only looks at how much variation in the dependent variable is explained, without checking whether the new features genuinely improve the model.  
As a result, adding **irrelevant or random columns** can make the R² value appear higher, giving a **false impression of a better model**.

In short:

> **More features ≠ Better model**, but R² may misleadingly suggest so.


In [64]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

In [65]:
# Demonstration of R² Score Problem
data = pd.read_csv('Experience-Salary.csv').sample(30)
df = data.copy()
X = data['exp(in months)'].values
Y = data['salary(in thousands)'].values
X_trans = X.reshape(-1, 1)


In [66]:
# In this example, we have a dataset containing Experience (in years) and Salary.  
model1 = LinearRegression()
model1.fit(X_trans, Y)
print('DataSet', data.head())
print(" \n R2-Score:-", r2_score(Y, model1.predict(X_trans)))
# Normally, we expect that as experience increases, 
# salary also increases — so R² should capture that relationship.

DataSet      exp(in months)  salary(in thousands)
726       20.432469             20.119180
644       18.308393             18.170322
535       27.547155             39.250930
500       20.080906             17.517721
66        29.338544             33.860652
 
 R2-Score:- 0.6636716354252508


In [67]:
# Now, we intentionally add an irrelevant column like 
# Day (values such as 'Mon', 'Tue', 'Wed', etc.) 
# or Gender like male and female, color , animql ect.
# # These column has no logical relation to salary.
days =  pd.Series(['sun','mon', 'tue', 'wen', 'thu', 'fri', 'sat'])
gender = pd.Series(['male', 'female'])
df.loc[:, 'days'] = [days[i % 7] for i in range(len(df))]
df.loc[:, 'gender'] = [gender[i % 2] for i in range(len(df))]
df['color'] = np.random.choice(['Red', 'Green', 'Blue', 'Yellow'], len(df))
df['animal'] = np.random.choice(['Dog', 'Cat', 'Cow', 'Goat'], len(df))
df = pd.get_dummies(df ,columns=['days', 'gender', 'color', 'animal'],drop_first=True, dtype=int)


In [72]:
df

,exp(in months),salary(in thousands),days_mon,days_sat,days_sun,days_thu,days_tue,days_wen,gender_male,color_Green,color_Red,color_Yellow,animal_Cow,animal_Dog,animal_Goat
726,20.432469,20.119180,0,0,1,0,0,0,1,0,0,1,0,1,0
644,18.308393,18.170322,1,0,0,0,0,0,0,0,1,0,0,0,0
535,27.547155,39.250930,0,0,0,0,1,0,1,0,0,0,0,0,0
500,20.080906,17.517721,0,0,0,0,0,1,0,0,0,0,0,0,1
66,29.338544,33.860652,0,0,0,1,0,0,1,0,0,1,0,0,1
336,37.716050,40.680969,0,0,0,0,0,0,0,0,0,0,1,0,0
210,37.370093,36.151112,0,1,0,0,0,0,1,0,0,0,1,0,0
41,21.880097,30.956644,0,0,1,0,0,0,0,1,0,0,0,1,0
185,22.059901,27.599797,1,0,0,0,0,0,1,0,0,1,1,0,0
785,23.376975,30.753132,0,0,0,0,1,0,0,0,0,0,0,1,0


In [69]:
# We then train a regression model and calculate the R² score again.
y = df['salary(in thousands)'].values
x = df.drop(columns='salary(in thousands)').values
model2 = LinearRegression()
model2.fit(x, y)
r2_score(y, model2.predict(x))
## 🧠 Expected Outcome
# Even though the new Day , Gender, color and animal column is completely useless, the R² score may increase or stay the same
# instead of decreasing.This happens because R² never penalizes a model for adding more independent variables   
# it only checks whether the model fits the data better numerically, even by chance.
# Hence, R² can give a false impression that the model improved.

0.832463204583086

# Adjusted R² — Fixing the Problem with R²

The **Adjusted R²** score is an improved version of R² that penalizes the model for adding **irrelevant or unnecessary features**.

While R² is calculated as:

$$
R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}
$$

The **Adjusted R²** is given by:

$$
R^2_{adj} = 1 - \left( \frac{(1 - R^2)(n - 1)}{n - k - 1} \right)
$$

Where:  
- \( n \) = number of data points  
- \( k \) = number of independent variables (features)

---

## 🧠 Why It’s Better

- When you **add a new feature**, Adjusted R² checks if it **actually improves the model**.  
- If the new feature **does not** help, Adjusted R² will **decrease** — unlike regular R², which always stays the same or increases.  
- Thus, it provides a **fairer and more realistic** measure of model performance.

---

### 🔍 Intuitive Understanding

> R² says: “More variables? Great! I’m happier now!” 😃  
>  
> Adjusted R² says: “Wait — did that new variable really help?  
> If not, I’ll reduce your score.” 😏

---

### ✅ Key Takeaway

- R² can **mislead** by increasing with irrelevant features.  
- **Adjusted R² fixes this** by adding a penalty term that depends on the number of predictors.  
- A **higher Adjusted R²** truly means a **better, more efficient model**.


In [70]:
def Adjusted_R2(R2, num_columns, num_rows):
    num = (1 - R2) * (num_rows - 1)
    deno = (num_rows - num_columns - 1)
    return 1 - (num / deno)
Adjusted_R2(r2_score(y, model2.predict(x)), x.shape[1], x.shape[0])

0.676095528860633

In [71]:
pd.DataFrame({
    "R2-Score":[r2_score(y, model2.predict(x))],
    "Adjusted R2-Score":[Adjusted_R2(r2_score(y, model2.predict(x)), x.shape[1], x.shape[0])]})

,R2-Score,Adjusted R2-Score
0,0.832463,0.676096


# 🎯 Conclusion: The Illusion of Accuracy

In this experiment, we observed a critical flaw in using **$R^2$ Score** alone for model evaluation, specifically when dealing with multiple features.

### 📊 The Scorecard
| Metric | Value | Interpretation |
| :--- | :--- | :--- |
| **$R^2$ Score** | **0.8325** | *Looks impressive!* Suggests the model explains **83%** of the variance. |
| **Adjusted $R^2$** | **0.6761** | *The Reality Check.* The true explanatory power is only **67%**. |

### 🧐 What Happened?
Our model's $R^2$ score of **0.83** is misleading. It was artificially inflated because we added irrelevant features (noise). The standard $R^2$ metric rewards complexity—it increases (or stays the same) with every new feature, regardless of whether that feature actually helps predict the target.

**Adjusted $R^2$**, however, imposed a penalty for those useless variables. It dropped the score to **0.67**, stripping away the "luck" factor and revealing the model's actual performance.

### 🛠️ Implication for Preprocessing
This distinction is vital during the **Feature Selection** and **Preprocessing** phases of a machine learning pipeline:
* **Quality > Quantity:** Adding more columns doesn't mean a better model.
* **Filter Noise:** We must rigorously remove features that do not contribute to the model to avoid overfitting.
* **Trust the Right Metric:** When comparing models with different numbers of features, **always rely on Adjusted $R^2$** over standard $R^2$.

### 💡 Final Verdict
> *" $R^2$ tells you how well your model fits the data, including the noise. Adjusted $R^2$ tells you how well your model actually generalizes. In the pursuit of a robust model, **Adjusted $R^2$ is the truth-teller we must listen to.** "*